In this tutorial, we will be using the Generally Nuanced Deep Learning Framework (GaNDLF) to perform training and inference on a VGG model with PathMNIST, a dataset of colon pathology images. This is a multi-class classification task: there are 9 different types of colon tissue displayed in the pathology images, each represented by its own class.

The VGG model is a CNN architecture mostly known for simplicity and effectiveness in image classification and has been used in object recognition, face detection, and medical image analysis. 
Its most notable features include its use of 16-19 weight layers and small 3x3 filters which allow for better performance and faster training times.



![VGG Model Image](https://d3i71xaburhd42.cloudfront.net/dae981902b1f6d869ef2d047612b90cdbe43fd1e/2-Figure1-1.png)

Figure 1 - Display of VGG Architecture - Visualizing and Comparing AlexNet and VGG using Deconvolutional Layers (W. Yu)

For more information on the model, refer to the paper Very Deep Convolutional Networks for Large-Scale Image Recognition by Karen Simonyan and Andrew Zisserman


This tutorial demonstrates how to use GaNDLF with a simple classification task. Some steps that would ordinarily be part of the workflow (e.g. data CSV and config YAML file construction) have already been performed for simplicity; please refer to the GaNDLF documentation (located at https://cbica.github.io/GaNDLF/) for more information regarding replication of these steps.

Command-Line Interface Note: Make sure you have Python installed before proceeding! Visit python.org/downloads for instructions on how do to this. Downloading version 3.7 is sufficient to complete this tutorial since this aligns with the version in Google Colab. 

Google Colab Note: Before continuing with this tutorial, please ensure that you are connected to the GPU by navigating to Runtime --> Change Runtime Type --> Hardware Accelerator and verifying that "GPU" is listed as the selected option. If not, it is highly recommended that you switch to it now. Also, if available, select the "High-RAM" option under Runtime → Change Runtime Type → Runtime Shape. Without this option selected, you may end up running out of RAM during training on a base notebook. 

Error Note: However, an error message may come up that says “You are connected to the GPU Runtime, but not utilizing the GPU.” This causes the program to stop when training the model because the program runs out of RAM.


Let's get started! First, we will clone the GaNDLF repo.

Google Colab Note: If you are following these steps on Google Colab, replace 0.0.16 in the line of code below with 0.0.14, as  the default version of Python in Colab (3.7) is not supported by the NumPy version used in the current version of GaNDLF.


In [ ]:
!pip install gandlf

The -b option indicates which branch we want to clone, and –depth 1 specifies the number of commits(or versions of the repository) that we want to retrieve. In this case, we are only retrieving one to save space and time, and since we are not making any changes to the actual GaNDLF code, this is sufficient.

Let's navigate to the newly created GaNDLF directory using the cd(change directory) command.


In [ ]:
%cd GaNDLF


Now, we'll install the appropriate version of PyTorch for use with GaNDLF. Pytorch is a machine learning framework primarily used for deep learning, and we will be using it here.


In [ ]:
!pip3 install torch==1.8.2 torchvision==0.9.2 torchaudio==0.8.2 --extra-index-url https://download.pytorch.org/whl/lts/1.8/cu111


pip is Python’s package manager that allows us to easily download different Python packages from the internet. If you do not have pip installed, visit pip.pypa.io/en/stable/installation/ for instructions.


Google Colab Error Note: You might see an error that says:
​​ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts. torchtext 0.14.1 requires torch==1.13.1, but you have torch 1.8.2+cu111 which is incompatible.
The next few cells may also have similar errors of the same form mentioning “pip’s dependency resolver” and version incompatibility, but GaNDLF installation will still be successful despite this, as we will see when verifying the installation.

Next, let's install OpenVINO, which is used by GaNDLF to generate optimized models for inference.

OpenVINO is a toolkit developed by Intel that allows for easier optimization of deep learning models in situations where access to computing resources may be limited. The model optimizes computation and memory usage while providing hardware-specific optimizations. OpenVINO currently supports over 68 different image classification models, allowing for flexibility of use with GaNDLF.

Visit docs.openvino.ai for more information.


We’ll need to install Matplotlib v.3.5.0 for use in plotting our results, for this version includes all of the features that we will be using later on. Let's do that now!


In [ ]:
!pip install matplotlib==3.5.0


Matplotlib is a library that allows us to create different types of data visualizations in Python. We will go over some of the specific features when we utilize it later on. Visit matplotlib.org for more information.

Google Colab Note: To be able to use the newly installed version of Matplotlib for plotting, go ahead and click the small gray "RESTART RUNTIME" button in the output directly above this code cell, and then continue with this tutorial once the restart process is complete.


Now, ensure you  are still in the GaNDLF directory before proceeding. Otherwise, run the following command. 


In [ ]:
%cd GaNDLF


For the last of our GaNDLF-related installations, let's install all required packages for GaNDLF.


In [ ]:
!pip install -e .

-e is short for editable, and . indicates that we are installing from the current directory (GaNDLF). Essentially, we are installing the packages in an editable mode because this allows us to change the source code in the packages without having to redownload the packages after we make modifications. 

Now, let's use gandlf_verifyInstall to verify our GaNDLF installation. 

In [ ]:
!python ./gandlf_verifyInstall



If you see the message, “GaNDLF is ready” after executing the previous step, then all steps have been followed correctly thus far. Let's move on to collecting our data. First, we will install the MedMNIST package in order to obtain the PathMNIST data that we will be using.


In [ ]:
!pip install medmnist


The original MNIST(Modified National Institute of Standards in Technology) database is a database containing various images of handwritten digits, and people use this dataset to train computer programs to recognize digits. MNIST is often used as a benchmark, and researchers can test different machine learning algorithms on MNIST to evaluate their performance.
MedMNIST is a database of biomedical images with 18 different datasets. 

The 2D Datasets are:
1. PathMNIST(Colon Pathology images)
2. ChestMNIST(Chest X-Ray images)
3. DermaMNIST(Dermatoscope images)
4. OCTMNIST(Retinal OCT images)
5. PneumoniaMNIST(Chest X-Ray images)
6. RetinaMNIST(Fundus Camera images)
7. BreastMNIST(Breast Ultrasound images)
8. BloodMNIST(Blood Cell Microscope images)
9. TissueMNIST(Kidney Cortex Microscope images)
10. - 12. OrganAMNIST, OrganCMNIST, and Organ SMNIST (Abdominal CT images)

The 3D Datasets are:
1. OrganMNIST3D(Abdominal CT images)
2. NoduleMNIST3D(Chest CT images)
3. AdrenalMNIST3D(Shape from Abdominal CT images)
4. FractureMNIST3D(Chest CT images)
5. VesselMNIST3D(Shape from Brain MRA images)
6. SynapseMNIST3D(Electron Microscope images)

Here are some visualizations of the MedMNIST dataset. 



![MedMNIST visualizations](https://medmnist.com/assets/v2/imgs/overview.jpg)


Figure 2 - MedMNIST datasets - MedMNIST v2 - A large-scale lightweight benchmark for 2D and 3D biomedical image classification (J. Yang)


The data is stored in a 28×28 (2D) or 28×28×28 (3D) format, similar to the 28×28 size of the images in the original MNIST dataset. 

For more information on MedMNIST and its installation visit github.com/MedMNIST/MedMNIST/ or medmnist.com

For the purposes of this tutorial, we will be focusing on PathMNIST, the colon pathology images. The PathMNIST dataset has a total of 107,180 samples, and these images were taken from the study "Predicting survival from colorectal cancer histology slides using deep learning: A retrospective multicenter study," by Jakob Nikolas Kather, Johannes Krisam, et al.

We will look more closely at the images in this dataset soon. 



Now, let's import MedMNIST and verify the version number before we move on. 

Command-Line Interface Note: This is Python code, so to run this, we must enter the Python shell. To do this, type python or python3 in the command prompt and press enter. After executing these commands, type exit() or quit() to exit the Python shell.


In [ ]:
import medmnist
print(medmnist.__version__)

Time to load our data! Let's download all MedMNIST datasets to the root directory. In this tutorial, we will only be using the PathMNIST dataset; however, feel free to use any of the other datasets you see being downloaded below (which are also mentioned above) to try out GaNDLF for yourself!


In [ ]:
!python -m medmnist download


-m stands for module, and this command is saying that we want to access the medmnist module and run the download function from that module.

Before we continue, let's navigate back to the base directory.


In [ ]:
%cd ..


Now, let's save all PathMNIST pathology images within the dataset folder (located inside the medmnist directory) in PNG format for use in training and inference.
If you've already gone through this tutorial and are looking to try using a different MedMNIST dataset, simply change --flag=pathmnist to any of the other datasets that were downloaded above—it's as simple as that!

In [ ]:
!python -m medmnist save --flag=pathmnist --folder=medmnist/dataset/ --postfix=png

The --folder option specifies where to save the downloaded information, and the --postfix option specifies that the images should be saved in png format.

For this tutorial, we will be using the full PathMNIST dataset for training, validation and testing. However, to improve efficiency and save time, you may consider using a fraction of this dataset instead with GaNDLF. 

To download the full dataset:

Let's retrieve and download the train_path_full data CSV file within the dataset folder, which consists of ~90,000 images.


In [ ]:
!wget -O /content/medmnist/dataset/train_path_full.csv "https://app.box.com/index.php?rm=box_download_shared_file&shared_name=zzokqk7hjzwmjxvamrxotxu78ihd5az0&file_id=f_972380483494"

location where the file should be saved and what the new filename will be
. 
Let's retrieve and download the val_path_full data CSV file within the dataset folder, which is the full validation dataset consisting of ~10,000 images. 


In [ ]:
!wget -O /content/medmnist/dataset/val_path_full.csv "https://app.box.com/index.php?rm=box_download_shared_file&shared_name=bjoh6hn27l6ifqqtrs7w66za2hdwlu0a&file_id=f_990373191120"

For the last of the data CSV files, let's retrieve and download the test_path_full data CSV file within the dataset folder. This CSV file contains ~7200 individual images.


In [ ]:
!wget -O /content/medmnist/dataset/test_path_full.csv "https://app.box.com/index.php?rm=box_download_shared_file&shared_name=jjzoifpdly0pmkdaguy0cxbdfkig81eq&file_id=f_990374552591"

To download the “tiny” dataset:

Let's retrieve and download the train_path_tiny data CSV file within the dataset folder, which consists of ~4,000 images.


In [ ]:
!wget -O /content/medmnist/dataset/train_path_full.csv "https://app.box.com/index.php?rm=box_download_shared_file&shared_name=um4003lkrvyj55jm4a0jz7zsuokb0r8o&file_id=f_991821586980"

Let's retrieve and download the val_path_tiny data CSV file within the dataset folder, which consists of ~1,000 images. 


In [ ]:
!wget -O /content/medmnist/dataset/val_path_full.csv "https://app.box.com/index.php?rm=box_download_shared_file&shared_name=rsmff27sm2z34r5xso1jx8xix7nhfspc&file_id=f_991817441206"

For the last of the data CSV files, let's retrieve and download the test_path_full data CSV file within the dataset folder. This CSV file contains 500  individual images.


In [ ]:
!wget -O /content/medmnist/dataset/test_path_full.csv "https://app.box.com/index.php?rm=box_download_shared_file&shared_name=22lm0qfzk5luap72mtdpzx5l3ocflopa&file_id=f_991819617152"


Command-Line Interface Note: If you run into any issues when trying to download these files, you can copy and paste these links into your web browser to download the .csv files. Then, you can manually place them into the desired directory using your file manager.


Now, we will retrieve and download the config YAML file to the base directory. This file specifies important information to be used in training and inference (model and training parameters, data preprocessing specifications, etc.).

For the purposes of this tutorial, we have already constructed this file to fit our specific task, but for other tasks and experiments that you may want to run, this file will need to be edited to fit the required specifications of your experiment. However, the overall structure of this file will stay the same regardless of your task, so you should be able to get by by simply downloading and editing the config.yaml file we're using below for use in your own experiments.

Either way, we highly encourage you to download and take a look at the structure of this file before proceeding if you intend to use GaNDLF for your own experiments, as it will be the backbone of all tasks you use GaNDLF with in the future. The file contains comments explaining the various parameters and what they mean. If you plan on trying to use any of the other datasets, specifically the 3D ones, make sure to change the number of dimensions on line 11 of the file to 3. 


In [ ]:
!wget -O config.yaml "https://app.box.com/index.php?rm=box_download_shared_file&shared_name=hs0zwezggl4rxtzgrcaq86enu7qwuvqx&file_id=f_974251081617"

Finally, let's retrieve and download an updated copy of the gandlf_collectStats file to the base directory for use in plotting and visualizing our results. While an earlier version of this file is present in the GaNDLF repo, it is not suitable for classification tasks and has been modified for this tutorial to produce classification training and validation accuracy and loss plots. This file will be included in the GaNDLF repo in a future update, but for now, we will retrieve the updated file externally for use in this tutorial.


In [ ]:
!wget -O gandlf_collectStats_final "https://app.box.com/index.php?rm=box_download_shared_file&shared_name=avq6pvqg3uzsc4uzbklab66mad6eaik5&file_id=f_989875069231"

Let's visualize some sample images and their classes from the PathMNIST dataset.
Image classes for reference:
Class 0: Adipose
Class 1: Background
Class 2: Debris
Class 3: Lymphocytes
Class 4: Mucus
Class 5: Smooth Muscle
Class 6: Normal Colon Mucosa
Class 7: Cancer-Associated Stroma
Class 8: Colorectal Adenocarcinoma Epithelium

Before running the code below, make sure you enter the python shell again if you are using a command line interface.  


In [ ]:
#Step 1
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd


#Step 2
df_pathmnist = pd.read_csv('./medmnist/dataset/pathmnist.csv')


#Step 3
selected_images = [32, 36, 46, 13, 14, 8, 12, 18, 5, 6, 17, 0, 16, 3, 7, 10, 43, 45, 55, 1, 31, 41, 4, 9, 11]


#Step 4
fig, ax = plt.subplot_mosaic([
   ['img0', 'img1', 'img2', 'img3', 'img4'],
   ['img5', 'img6', 'img7', 'img8', 'img9'],
   ['img10', 'img11', 'img12', 'img13', 'img14'],
   ['img15', 'img16', 'img17', 'img18', 'img19'],
   ['img20', 'img21', 'img22', 'img23', 'img24']
], figsize=(15, 15))


#Step 5
for i in range(len(selected_images)):
    img = selected_images[i]
    filename = df_pathmnist.iloc[img]['train0_0.png']
    img_class = df_pathmnist.iloc[img]['0']


    path_img = mpimg.imread(f'./medmnist/dataset/pathmnist/{filename}')


    ax[f'img{i}'].imshow(path_img)
    ax[f'img{i}'].axis('off')
    ax[f'img{i}'].set_title(f'Class {img_class}')


#Step 6
plt.show()


Let’s quickly go over what exactly this code is doing.
1. We import all of the necessary libraries, such as matplotlib(which we discussed earlier) and pandas, which is a Python library that provides methods for data manipulation. 
2. The file paths for the PathMNIST images are read and stored in the df_pathmnist variable. 
3. We initialize the selected_images variable which stores an array of integers. These integers represent the indices of the images from the pathmnist dataset we want to display. You can also try changing some of these values to see the different displayed plots and see more images in the dataset. 
4. We create a 5x5 grid of subplots to display our 25 images.
5. We iterate through the selected_images array and read the corresponding image files. Then, we display the images in the proper subplots.
6. Finally, plt.show() displays the plot we have created. 

Your resulting plot should have an assortment of images from the PathMNIST set including images from all of class 0-8.

Let’s quickly go over what these different classes are by using the results above:

Class 0:  Adipose
Adipose tissue is just tissue that stores fat. This tissue is made up of adipocytes, or fat cells, which store energy.



Class 1: Background
The background class represents the non-tissue areas of the image that aren’t necessarily areas of interest.



Class 2: Debris
Class 2 contains images of miscellaneous non-living particles or materials found in the colon.



Class 3: Lymphocytes 
Class 4 contains images of lymphocytes. Lymphocytes are a type of white blood cell that play a significant role in the immune system.



Class 4: Mucus
Class 4 contains images of normal mucus that is found in healthy colons.




Class 5: Smooth Muscle
Class 5 contains images of smooth muscle tissue that is seen in the walls of the colon.



Class 6: Normal Colon Mucosa
Class 6 contains images of the colon mucosa, which lines the colon.




Class 7: Cancer-Associated Stroma
Class 7 contains images of stroma, which is supportive tissue that provides tumors with nutrients and protection to assist in the cancer’s survival.




Class 8: Colorectal Adenocarcinoma Epithelium
The epithelium is the tissue that lines the colon, and Class 8 contains images of epithelial cells affected by colorectal adenocarcinoma, a type of cancer.







Now, on to training! Since there is only one GPU, let's set CUDA_VISIBLE_DEVICES to 0 to train on the first (and only) available GPU. The way to execute this code will vary based on what you are running your code on.

On Colab run this Python code: 


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

If you are following these instructions on your own command line interface, instead run the following: 

export "CUDA_VISIBLE_DEVICES"="0"

Let's run the training script! For this run, we will pass in the config.yaml file that we just downloaded to the -c parameter, the training and validation CSV files to the -i parameter, and the model directory to the -m parameter (folder will automatically be created if it doesn't exist, which, in our case, it doesn't). We will also specify -t True to indicate that we are training and -d cuda to indicate that we will be training on the GPU. For demonstration purposes, we will only be training on 5 epochs. The number of epochs is the number of times that the model will pass through the entire training dataset during the training process. In our case, the model will pass through the dataset 5 times. 



In [ ]:
!python /content/GaNDLF/gandlf_run -c /content/config.yaml -i /content/medmnist/dataset/train_path_full.csv,/content/medmnist/dataset/val_path_full.csv -m /content/model/ -t True -d cuda



You will likely notice a persistent error saying "y_pred contains classes not in y_true"--this is a known issue and will not affect our training performance, so feel free to ignore it.

If your program stops for any reason and you try to run it again, you may see this error:
ValueError: The parameters are not the same as the ones stored in the previous run, please re-check.
To resolve this, delete the existing model by deleting the “model” file and then try executing the command again. 

Potential Google Colab Error: If your Google Colab notebook is not correctly using the GPU, the program may run out of RAM and stop during the process of constructing the queue for training data. 

Now that training is complete, let's collect and save model statistics to the output_stats folder. Using -c True indicates that we'd like the 4 plots ordinarily generated by this command to be combined into two plots by overlaying training and validation statistics on the same graphs instead of keeping them separate. Feel free to experiment with this command by using -c False instead and viewing the resulting plots.


In [ ]:
!python /content/gandlf_collectStats_final -m /content/model/ -o /content/output_stats -c True

Now, let's view our generated plots! Make sure to enter the Python shell if you are working in the command line interface.


In [ ]:
from IPython.display import Image
Image("/content/output_stats/plot.png", width=1500, height=1000)

Your results should show both an Accuracy Plot and a Loss Plot.

Since we only trained the model on a very small number of epochs, we shouldn't be expecting very impressive results here. However, from the graphs, we can tell that accuracy is steadily increasing and loss is steadily decreasing, which is a great sign.

Finally, let's run the inference script. This is almost identical to running the training script; however, note that the argument for the -t parameter has been changed from True to False to specify that we are not training, and we are using the test_path_full csv file to access the testing images.


In [ ]:
!python /content/GaNDLF/gandlf_run -c /content/config.yaml -i /content/medmnist/dataset/test_path_full.csv -m /content/model/ -t False -d cuda

Now that inference is complete, let's view some sample test images along with their predicted and ground truth classes to get a visual idea of how well our model did on each class. Remember to enter the Python shell if you are using the command line interface.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd


df_preds = pd.read_csv('./model/final_preds_and_avg_probs.csv')


selected_images = [4, 7, 14, 22, 26, 3, 11, 17, 37, 45, 52, 1, 2, 40, 47, 76, 8, 13, 28, 19, 36, 79, 0, 9, 10]


fig, ax = plt.subplot_mosaic([
   ['img0', 'img1', 'img2', 'img3', 'img4'],
   ['img5', 'img6', 'img7', 'img8', 'img9'],
   ['img10', 'img11', 'img12', 'img13', 'img14'],
   ['img15', 'img16', 'img17', 'img18', 'img19'],
   ['img20', 'img21', 'img22', 'img23', 'img24']
], figsize=(13, 13), constrained_layout = True)


for i in range(len(selected_images)):
    img = selected_images[i]
    filename = df_preds.iloc[img]['SubjectID']
    ground_truth = filename.split('_')[1].split('.')[0]
    pred_class = df_preds.iloc[img]['PredictedClass']

    path_img = mpimg.imread(f'./medmnist/dataset/pathmnist/{filename}')


    ax[f'img{i}'].imshow(path_img)
    ax[f'img{i}'].axis('off')
    ax[f'img{i}'].set_title(f'Predicted Class: {pred_class}\nGround                .     Truth: {ground_truth}')


plt.show()



Your produced plot should contain 16 images from the PathMNIST set, and each image should be accompanied by both a Predicted Class and a Ground Truth label. 

We can see that our model did a decent job of making predictions. However, we can see that it has a few common misconceptions. We can see that the model mistook smooth muscle for debris thrice and cancer-associated stroma for debris once. The model also mistook mucus as adipose twice, and it mistook Colorectal Adenocarcinoma Epithelium as Normal Colon Mucosa once.


To conclude this tutorial, let's zoom out and take a look at how well our model did as a whole on each class by constructing a confusion matrix from our inference data.
Note: if you'd like, feel free to change the colormap of the confusion matrix (denoted by "cmap" in the cm_display.plot() command) to your liking. Here's a list of some of the most popular colormaps: viridis (default), plasma, inferno, magma, cividis.

Execute the following code in the Python shell: 


In [ ]:
import sklearn
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


gt_list = []
pred_list = []


for i in range(len(df_preds)):
    filename = df_preds.iloc[i]['SubjectID']


    ground_truth = int(filename.split('_')[1].split('.')[0])
    pred_class = int(df_preds.iloc[i]['PredictedClass'])


    gt_list.append(ground_truth)
    pred_list.append(pred_class)


cm = confusion_matrix(gt_list, pred_list)
cm_display = ConfusionMatrixDisplay(confusion_matrix = cm)


fig, ax = plt.subplots(figsize = (12, 12))
cm_display.plot(cmap = 'viridis', ax = ax)
plt.show()


Let’s quickly go over this code:

1. We import the Scikit-Learn library, and we will be using this library's methods of creating confusion matrices. 

2. We create empty lists gt_list and pred_list to store our ground truth and predicted labels respectively.

3. The for loop iterates through df_preds and extracts the ground truth label and the predicted class for each image and then appends the labels to their corresponding lists.

4. We create a confusion matrix from these two lists and then call the plot() method to visualize it with our desired colormap.

5. We display the confusion matrix using plt.show().


Your resulting plot should be a 9x9 matrix that displays Predicted label vs. True label

In our matrix, larger numbers along the diagonal represent correct classifications. Here, we can see that while the model performed well overall, it had difficulties when it came to images of Class 7, incorrectly predicting a majority of them as belonging to Class 2 instead. We can see a similar trend with images of Class 5, with the model incorrectly predicting most of them as belonging to Class 2.

Given the appearance of the accuracy and loss plots, had we trained on more epochs, we would have expected these results to improve. However, given that we only trained on 5 epochs, these are great results. Indeed, the model did very well on other classes, including Classes 0 and 1.

That concludes this GaNDLF tutorial! Hopefully, this tutorial was helpful to you in understanding how GaNDLF works as well as how to apply it to your own projects. If you need any additional information about GaNDLF's usage and capabilities, please consult the GitHub repo (https://github.com/CBICA/GaNDLF) and the documentation (https://cbica.github.io/GaNDLF/). For more questions and support, please visit the Discussions page on GitHub (https://github.com/CBICA/GaNDLF/discussions)

